In [1]:
import os
import re
from bs4 import BeautifulSoup

In [2]:
# Caminho para a pasta de descompactados ASC
pasta_conversa = ['/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Fevereiro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Abril', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Maio', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Junho', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Julho'
]

def extrair_anexos_arquivo(caminho_arquivo):
    anexos_encontrados = []
    try:
        with open(caminho_arquivo, 'r', encoding='latin-1') as file:
            contents = file.read()
        soup = BeautifulSoup(contents, 'html.parser')

        # Encontrar todos os links para anexos
        links_anexos = soup.find_all('a', href=True)

        for link in links_anexos:
            href = link['href']
            if 'anexos/' in href:
                caminho_anexo_origem = os.path.join(os.path.dirname(caminho_arquivo), href)
                anexos_encontrados.append(caminho_anexo_origem)

    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo {caminho_arquivo}: {str(e)}")

    return anexos_encontrados

def extrair_informacoes_arquivo(caminho_arquivo):
    try:
        with open(caminho_arquivo, 'r', encoding='latin-1') as file:
            contents = file.read()
        soup = BeautifulSoup(contents, 'html.parser')

        info_element = soup.find('p')
        if info_element:
            info_text = info_element.get_text(strip=True)
        else:
            info_text = ""

        mensagens = []
        mensagem_elements = soup.find_all('li', class_=["agente", "contato"])
        for mensagem in mensagem_elements:
            time_element = mensagem.find('span', class_='time')
            if time_element:
                time = time_element.get_text(strip=True)
                mensagem_text_element = mensagem.find('p')
                if mensagem_text_element:
                    mensagem_text = mensagem_text_element.get_text(strip=True)
                    mensagens.append({"horario": time, "mensagem": mensagem_text, 'arquivo': caminho_arquivo})

        return {"info_conversa": info_text, "mensagens": mensagens}
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo {caminho_arquivo}: {str(e)}")
        return {"info_conversa": "", "mensagens": []}

def processar_pasta(pasta_conversas):
    print("Procurando em:", pasta_conversas)
    arquivos_html = [os.path.join(pasta_conversas, arquivo) for arquivo in os.listdir(pasta_conversas) if arquivo.endswith('.html')]
    
    json_data = set()  
    for arquivo_html in arquivos_html:
        encontrou_correspondencia = False  
        
        informacoes_arquivo = extrair_informacoes_arquivo(arquivo_html)
        for mensagem in informacoes_arquivo["mensagens"]:
            texto = mensagem["horario"] + mensagem["mensagem"]
            if re.search(r'BRUNO MENDES SOARES|Bruno Mendes Soares|FGS7E66|fgs7e66|063(\.|)095(\.|)105-50', texto, re.I): 
                encontrou_correspondencia = True
                json_data.add(mensagem['arquivo'])
                # Print do arquivo encontrado
                print(mensagem['arquivo'])
                #print("Horário:", mensagem["horario"])
                #print("Mensagem:", mensagem["mensagem"])
                print()
                
                # Listar caminhos dos anexos encontrados apenas nas conversas correspondentes
                anexos_arquivo = extrair_anexos_arquivo(arquivo_html)
                for anexo in anexos_arquivo:
                    print("Caminho do anexo:", anexo)
                
                break  
                
        if encontrou_correspondencia:
            continue  
        
    json_data_new = list(json_data)
    for item in json_data_new:
        print(item)

for pasta in pasta_conversa:
    processar_pasta(pasta)


Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Fevereiro
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco
/Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/1165282-202301165282-20230811.html

Caminho do anexo: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/anexos/MODELOFOTOS-892860001679663908.jpg
Caminho do anexo: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/anexos/02D8D0F303EB7EF3AF333614F4FAD08A.pdf
Caminho do anexo: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/anexos/D838FD082F28B1E71079323BF42F7811.jpeg
Caminho do anexo: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/anexos/59C1F110507706B90F02762BAC7CC06E.jpeg
Caminho do anexo: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/anexos/0992A31C23DCC93682334D143D3C9B08.jpeg
Caminho do anexo: /Volumes/Backup/ASC_Atendimentos_Descompactados/2023/Marco/anexos/7EEA64D5D40E0F58E0DFF4645EDDC189.jpeg
Caminho do anexo: /Volume